In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

_HASH_BUCKET_SIZE = 1000

categorical_features = ['student_id', 'card_id', 'timeslot', 'trans_type', 'category', 'device_name']

continus_features = ['amount', 'remained_amount']

df = pd.read_csv('../data/consum_cleaned.csv')
df[categorical_features] = df[categorical_features].fillna('')
df[continus_features] = df[continus_features].fillna(0)
df['card_id'] = df['card_id'].apply(str)
_CSV_COLUMNS = df.columns.tolist()

TOTAL_NUM = df.shape[0]
FIFTEEN_PCT = int(TOTAL_NUM * 0.15)
_NUM_EXAMPLES = {
    'train': TOTAL_NUM - FIFTEEN_PCT * 2,
    'validation': FIFTEEN_PCT,
}
_ID_SIZE = df.student_id.drop_duplicates().count()
_LABEL_NUM = 104

def build_model_columns():
    """Builds a set of wide and deep feature columns."""
    # Continuous variable columns
    cate_columns = []
    cate_embedding_columns = []
    cate_no_embedding_columns = []
    continus_columns = []
    embedding_features = []
    for col in categorical_features:
        coluniquelen = len(df[col].unique())
        if coluniquelen > _HASH_BUCKET_SIZE:
            print("col name :{}\t\tcount:{}".format(col, coluniquelen))
            embedding_features.append(col)
    for feat in categorical_features:
        f = tf.feature_column.categorical_column_with_vocabulary_list(feat, df[feat].unique())
        if feat in embedding_features:
            f1 = tf.feature_column.embedding_column(f, dimension=100)
            cate_embedding_columns.append(f1)
        else:
            f1 = tf.feature_column.indicator_column(f)
            cate_no_embedding_columns.append(f1)
        cate_columns.append(f)
    for feat in continus_features:
        f = tf.feature_column.numeric_column(feat)
        continus_columns.append(f)
    deep_columns = cate_embedding_columns + cate_no_embedding_columns + continus_columns
    return deep_columns


def input_fn(data_file, num_epochs, shuffle, batch_size, is_classification=False):
    """Generate an input function for the Estimator."""
    assert tf.gfile.Exists(data_file), (
        '%s not found. Please make sure you have run census_dataset.py and '
        'set the --data_dir argument to the correct path.' % data_file)

    def parse_csv(value):
        tf.logging.info('Parsing {}'.format(data_file))
        record_defaults = [[0.0] if c in continus_features else [''] for c in _CSV_COLUMNS]
        columns = tf.decode_csv(value, record_defaults)
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop('placei')
        if is_classification:
            classes = tf.strings.to_number(labels, out_type=tf.int64, name='label_class')
            return features, classes
        else:
            return features, labels

    # Extract lines from input files using the Dataset API.
    dataset = tf.data.TextLineDataset(data_file)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

    dataset = dataset.map(parse_csv, num_parallel_calls=5)

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
    # dataset = dataset.batch(batch_size)
    return dataset


In [9]:
deep_columns = build_model_columns()
train_file = '../data/tf_train.csv'
dataset = input_fn(train_file, 1, False, 10)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
v = sess.run(one_element)
x = tf.feature_column.input_layer(v[0], deep_columns)
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
init_table = tf.tables_initializer()
sess.run(init_op)
sess.run(init_table)
sess.run(x)

col name :student_id		count:41633
col name :card_id		count:62582
INFO:tensorflow:Parsing ../data/tf_train.csv


AttributeError: 'BatchDataset' object has no attribute 'values'

In [49]:
init_op = tf.global_variables_initializer()
init_table = tf.tables_initializer()
sess.run(init_op)
sess.run(init_table)

FailedPreconditionError: Table already initialized.
	 [[{{node input_layer/category_indicator/category_lookup/hash_table/table_init}} = InitializeTableV2[Tkey=DT_STRING, Tval=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_layer/category_indicator/category_lookup/hash_table, input_layer/category_indicator/category_lookup/Const, input_layer/category_indicator/category_lookup/ToInt64)]]

Caused by op 'input_layer/category_indicator/category_lookup/hash_table/table_init', defined at:
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    a

In [35]:
np.set_printoptions(suppress=True) # don't use scientific notation
a[0]

array([ 0.5       , -0.00542771,  0.03267819, -0.06834557, -0.1618337 ,
        0.06931791, -0.11821896,  0.03961104,  0.06839292,  0.18072768,
        0.13669781,  0.10483644,  0.02830074,  0.01686157,  0.00739236,
        0.082309  , -0.00197054, -0.07846905, -0.08892803,  0.09540547,
       -0.0158    ,  0.04721189, -0.0667192 , -0.14977056, -0.03364233,
        0.0102575 , -0.16489199, -0.03690752, -0.17593393, -0.06212981,
        0.04176848, -0.04514671,  0.05780005,  0.13186991,  0.01725243,
       -0.05645569, -0.13648224,  0.1867404 , -0.0612515 , -0.01681192,
       -0.03353823, -0.17015332,  0.10795977, -0.06483619, -0.00929453,
       -0.00234832, -0.07309407,  0.03676496, -0.13697648, -0.1901229 ,
        0.07651262,  0.06804652, -0.10970445,  0.04430771, -0.04081658,
        0.00203471,  0.03239926, -0.05470664, -0.04123081, -0.10199185,
        0.02632782,  0.12340486, -0.05761214,  0.03649587, -0.19984211,
       -0.04981123, -0.02424479, -0.08442622, -0.07562018, -0.05

In [41]:
v

({'student_id': array([b'2012059140024', b'2012059100025', b'201322030644',
         b'201522020746', b'2014080102005', b'201522060723',
         b'201321010125', b'201521037019', b'2012060090018',
         b'201522035005'], dtype=object),
  'card_id': array([b'175928', b'157006', b'201045', b'326667', b'302391', b'329723',
         b'306110', b'331421', b'300824', b'327739'], dtype=object),
  'amount': array([ 0.5 , -3.12,  0.1 ,  0.1 ,  0.3 ,  0.6 ,  0.2 ,  0.4 ,  0.6 ,
          0.1 ], dtype=float32),
  'remained_amount': array([ 24.61,  85.63,  19.97, 120.98,  57.32,  83.36, 499.4 ,  52.7 ,
          72.26,  94.53], dtype=float32),
  'trans_type': array([b'POS\xe6\xb6\x88\xe8\xb4\xb9', b'POS\xe6\xb6\x88\xe8\xb4\xb9',
         b'POS\xe6\xb6\x88\xe8\xb4\xb9', b'POS\xe6\xb6\x88\xe8\xb4\xb9',
         b'POS\xe6\xb6\x88\xe8\xb4\xb9', b'POS\xe6\xb6\x88\xe8\xb4\xb9',
         b'POS\xe6\xb6\x88\xe8\xb4\xb9', b'POS\xe6\xb6\x88\xe8\xb4\xb9',
         b'POS\xe6\xb6\x88\xe8\xb4\xb9', b'POS\xe6

In [52]:
tensor = tf.feature_column.input_layer(v[0], [deep_columns[0]])
init_op = tf.global_variables_initializer()
init_table = tf.tables_initializer()
sess.run(init_op)
# sess.run(init_table)
sess.run(tensor[0])

FailedPreconditionError: Table not initialized.
	 [[{{node input_layer_7/student_id_embedding/hash_table_Lookup}} = LookupTableFindV2[Tin=DT_STRING, Tout=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_layer_7/student_id_embedding/student_id_lookup/hash_table, input_layer_7/student_id_embedding/to_sparse_input/values, input_layer_7/student_id_embedding/student_id_lookup/hash_table/Const)]]

Caused by op 'input_layer_7/student_id_embedding/hash_table_Lookup', defined at:
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()


In [53]:
sess.run(init_table)

FailedPreconditionError: Table already initialized.
	 [[{{node input_layer_3/student_id_embedding/student_id_lookup/hash_table/table_init}} = InitializeTableV2[Tkey=DT_STRING, Tval=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_layer_3/student_id_embedding/student_id_lookup/hash_table, input_layer_3/student_id_embedding/student_id_lookup/Const, input_layer_3/student_id_embedding/student_id_lookup/ToInt64)]]

Caused by op 'input_layer_3/student_id_embedding/student_id_lookup/hash_table/table_init', defined at:
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/liwenjie/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/liwenjie/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line

In [54]:
tensor

<tf.Tensor 'input_layer_7/concat:0' shape=(10, 100) dtype=float32>